<a href="https://colab.research.google.com/github/RPAlbuquerque/PhD-Dissertation---Paper-2/blob/main/Mobility_Paper_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##🔧 Bloco 1: Setup & Imports

In [ ]:
# Title: Setup and Configuration
import pandas as pd
import numpy as np
import os
import gzip
import sys
import multiprocessing as mp
from tqdm import tqdm

# Ensure real-time output in Jupyter
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea{font-size: 14px;}</style>"))

# Path to raw mobility data
MOBILITY_PATH = "/n/netscratch/cga/Lab/rpalbuquerque/Mobility/"
MOBILITY_FILE = os.path.join(MOBILITY_PATH, "mobilidade_unificada_Aug_2024.csv.gz")

# Output folder for intermediate files
OUTPUT_PATH = os.path.join(MOBILITY_PATH, "processed_aug2024")
os.makedirs(OUTPUT_PATH, exist_ok=True)

print("Environment set up. Ready to load mobility data.")

Environment set up. Ready to load mobility data.


In [ ]:
##📥 Block 2: Reading and pre visualization

In [ ]:
# Title: Load Mobility Data (August 2024)

chunks = []
with gzip.open(MOBILITY_FILE, 'rt') as f:
    first_chunk = pd.read_csv(f, nrows=100_000)
    print("Preview of mobility data (first 5 rows):")
    display(first_chunk.head())
    print(f"\nColumns: {list(first_chunk.columns)}")

Preview of mobility data (first 5 rows):


,month_part,store_id,demographics_gender,demographics_age_range,demographics_class,...,dow_7_h1,dow_7_h2,dow_7_h3,dow_7_h4,state
0,2024-08-01T00:00:00.000Z,210090705000051,F,18_24,D,...,4.30,11.95,6.71,15.20,MA
1,2024-08-01T00:00:00.000Z,210550005000065,F,30_39,D,...,3.29,0.89,0.00,1.25,MA
2,2024-08-01T00:00:00.000Z,211170605000059,M,30_39,D,...,0.00,0.00,0.00,0.00,MA
3,2024-08-01T00:00:00.000Z,210140005000172,M,70_79,E,...,270.66,325.01,286.06,231.58,MA
4,2024-08-01T00:00:00.000Z,210005505000002,M,18_24,E,...,588.99,843.53,664.34,998.97,MA



Columns: ['month_part', 'store_id', 'demographics_gender', 'demographics_age_range', 'demographics_class', 'unique', 'visits', 'raw_unique', 'raw_visits', 'unique_q1', 'visits_q1', 'unique_q2', 'visits_q2', 'unique_q3', 'visits_q3', 'unique_q4', 'visits_q4', 'repeat_q1', 'repeat_q2', 'repeat_q3', 'repeat_q4', 'repeat_visitors', 'new_visitor_q1', 'new_visitor_q2', 'new_visitor_q3', 'new_visitor_q4', 'new_visitors', 'dwell_time_mins_q1', 'dwell_time_mins_q2', 'dwell_time_mins_q3', 'dwell_time_mins_q4', 'dwell_time_mins', 'dow_1_h1', 'dow_1_h2', 'dow_1_h3', 'dow_1_h4', 'dow_2_h1', 'dow_2_h2', 'dow_2_h3', 'dow_2_h4', 'dow_3_h1', 'dow_3_h2', 'dow_3_h3', 'dow_3_h4', 'dow_4_h1', 'dow_4_h2', 'dow_4_h3', 'dow_4_h4', 'dow_5_h1', 'dow_5_h2', 'dow_5_h3', 'dow_5_h4', 'dow_6_h1', 'dow_6_h2', 'dow_6_h3', 'dow_6_h4', 'dow_7_h1', 'dow_7_h2', 'dow_7_h3', 'dow_7_h4', 'state']


In [ ]:
##⚙️ Bloco 3: Clean and Save Raw Mobility

In [ ]:
# Title: High-Efficiency Chunked Cleaning and Direct Save

import pandas as pd
import os

MOBILITY_FILE = "/n/netscratch/cga/Lab/rpalbuquerque/Mobility/mobilidade_unificada_Aug_2024.csv.gz"
OUTPUT_PATH = "/n/netscratch/cga/Lab/rpalbuquerque/Mobility/processed_aug2024"
os.makedirs(OUTPUT_PATH, exist_ok=True)
clean_output_path = os.path.join(OUTPUT_PATH, "mobility_aug2024_clean.csv.gz")

chunk_size = 2_000_000
row_count = 0
is_first = True

print("⏳ Starting streaming read, clean, and save...")

for chunk in pd.read_csv(MOBILITY_FILE, compression='gzip', low_memory=False, chunksize=chunk_size):
    initial_rows = len(chunk)
    chunk = chunk.dropna(subset=['store_id', 'month_part'])
    row_count += len(chunk)

    # Write to disk incrementally
    chunk.to_csv(clean_output_path, mode='w' if is_first else 'a',
                 header=is_first, index=False, compression='gzip')
    is_first = False

    print(f"✔️ Processed chunk: {initial_rows} → {len(chunk)} rows written")

print(f"\n✅ Total rows after cleaning: {row_count:,}")
print(f"✅ Final cleaned file saved at: {clean_output_path}")

⏳ Starting streaming read, clean, and save...
✔️ Processed chunk: 2000000 → 2000000 rows written
✔️ Processed chunk: 2000000 → 2000000 rows written
✔️ Processed chunk: 2000000 → 2000000 rows written
✔️ Processed chunk: 2000000 → 2000000 rows written
✔️ Processed chunk: 2000000 → 2000000 rows written
✔️ Processed chunk: 2000000 → 2000000 rows written
✔️ Processed chunk: 2000000 → 2000000 rows written
✔️ Processed chunk: 629714 → 629714 rows written

✅ Total rows after cleaning: 14,629,714
✅ Final cleaned file saved at: /n/netscratch/cga/Lab/rpalbuquerque/Mobility/processed_aug2024/mobility_aug2024_clean.csv.gz


In [ ]:
##📊 Block 4 — Aggregation by setor with high performance

In [ ]:
# Title: Ultra-Fast Aggregation by Sector (Vectorized Version)

import pandas as pd
import os

clean_input_path = "/n/netscratch/cga/Lab/rpalbuquerque/Mobility/processed_aug2024/mobility_aug2024_clean.csv.gz"
agg_output_path = "/n/netscratch/cga/Lab/rpalbuquerque/Mobility/aggregated_aug2024"
os.makedirs(agg_output_path, exist_ok=True)
agg_file = os.path.join(agg_output_path, "mobility_by_sector.csv.gz")

chunk_size = 2_000_000
agg_chunks = []

print("⏳ Starting vectorized aggregation by store_id...")

for chunk in pd.read_csv(clean_input_path, chunksize=chunk_size, low_memory=False):
    chunk_grouped = chunk.groupby('store_id').agg({
        'unique': 'sum',
        'visits': 'sum',
        'raw_unique': 'sum',
        'raw_visits': 'sum',
        'repeat_visitors': 'sum',
        'new_visitors': 'sum',
        'dwell_time_mins': ['sum', 'count'],
        'state': 'first'
    })
    agg_chunks.append(chunk_grouped)

# Combine all intermediate results
combined = pd.concat(agg_chunks).groupby('store_id').agg({
    ('unique', 'sum'): 'sum',
    ('visits', 'sum'): 'sum',
    ('raw_unique', 'sum'): 'sum',
    ('raw_visits', 'sum'): 'sum',
    ('repeat_visitors', 'sum'): 'sum',
    ('new_visitors', 'sum'): 'sum',
    ('dwell_time_mins', 'sum'): 'sum',
    ('dwell_time_mins', 'count'): 'sum',
    ('state', 'first'): 'first'
})

# Flatten columns
combined.columns = [
    'total_unique_visitors', 'total_visits', 'raw_unique_visitors', 'raw_total_visits',
    'total_repeat_visitors', 'total_new_visitors', 'dwell_sum', 'dwell_count', 'state'
]
combined.reset_index(inplace=True)
combined['avg_dwell_time_mins'] = combined['dwell_sum'] / combined['dwell_count']

# Final format
final_df = combined[[
    'store_id', 'total_unique_visitors', 'total_visits', 'raw_unique_visitors',
    'raw_total_visits', 'total_repeat_visitors', 'total_new_visitors',
    'avg_dwell_time_mins', 'state'
]].rename(columns={'store_id': 'code_censo'})

# Save final file
final_df.to_csv(agg_file, index=False, compression='gzip')
print(f"✅ Aggregated file saved at: {agg_file}")

⏳ Starting vectorized aggregation by store_id...
✅ Aggregated file saved at: /n/netscratch/cga/Lab/rpalbuquerque/Mobility/aggregated_aug2024/mobility_by_sector.csv.gz


In [ ]:
## ⚙️ Block 5: Full Aggregation of All Numeric Columns by Sector

In [ ]:
# Title: Full Aggregation of All Numeric Columns by Sector (No Re-Groupby)

import pandas as pd
import numpy as np
import os

# Paths
clean_input_path = "/n/netscratch/cga/Lab/rpalbuquerque/Mobility/processed_aug2024/mobility_aug2024_clean.csv.gz"
agg_output_path = "/n/netscratch/cga/Lab/rpalbuquerque/Mobility/aggregated_aug2024"
os.makedirs(agg_output_path, exist_ok=True)
agg_file = os.path.join(agg_output_path, "mobility_by_sector_full.csv.gz")

# Config
chunk_size = 2_000_000
aggregated_chunks = {}

print("⏳ Starting robust full numeric aggregation by sector...")

for chunk in pd.read_csv(clean_input_path, chunksize=chunk_size, low_memory=False):
    numeric_cols = chunk.select_dtypes(include=[np.number]).columns.tolist()
    grouped = chunk.groupby("store_id")[numeric_cols].agg(['sum', 'mean'])

    # Aggregate into dict
    for store_id, row in grouped.iterrows():
        if store_id not in aggregated_chunks:
            aggregated_chunks[store_id] = row
        else:
            aggregated_chunks[store_id] += row

# Convert back to DataFrame
final = pd.DataFrame.from_dict(aggregated_chunks, orient='index')
final.index.name = 'code_censo'
final.reset_index(inplace=True)

# Flatten MultiIndex columns
final.columns = ['code_censo'] + [f"{col}_{stat}" for col, stat in final.columns[1:]]

# Save to disk
final.to_csv(agg_file, index=False, compression='gzip')
print(f"✅ Full numeric aggregation saved at: {agg_file}")

⏳ Starting robust full numeric aggregation by sector...
✅ Full numeric aggregation saved at: /n/netscratch/cga/Lab/rpalbuquerque/Mobility/aggregated_aug2024/mobility_by_sector_full.csv.gz


In [ ]:
##⚙️ Bloco 6: Sociodemographic Aggregation by Sector

In [ ]:
# Title: Sociodemographic Aggregation by Sector (Proportions)

import pandas as pd
import os
from collections import defaultdict

# Paths
clean_input_path = "/n/netscratch/cga/Lab/rpalbuquerque/Mobility/processed_aug2024/mobility_aug2024_clean.csv.gz"
agg_output_path = "/n/netscratch/cga/Lab/rpalbuquerque/Mobility/aggregated_aug2024"
os.makedirs(agg_output_path, exist_ok=True)
output_file = os.path.join(agg_output_path, "mobility_sociodemographics.csv.gz")

# Config
chunk_size = 2_000_000
agg_data = defaultdict(lambda: defaultdict(int))

print("⏳ Starting sociodemographic aggregation...")

for chunk in pd.read_csv(clean_input_path, chunksize=chunk_size, low_memory=False):
    for var in ['demographics_gender', 'demographics_age_range', 'demographics_class']:
        grouped = chunk.groupby('store_id')[var].value_counts()
        for (store_id, category), count in grouped.items():
            col_name = f"{var}_{category}"
            agg_data[store_id][col_name] += count

print("✅ Aggregation done. Converting to DataFrame...")

# Transform to DataFrame
records = []
for store_id, counts in agg_data.items():
    total = sum(counts.values())
    row = {'code_censo': store_id}
    for col, val in counts.items():
        row[col] = val / total  # proportion
    records.append(row)

final_df = pd.DataFrame(records)
final_df.to_csv(output_file, index=False, compression='gzip')
print(f"✅ Sociodemographic proportions saved at: {output_file}")

⏳ Starting sociodemographic aggregation...
✅ Aggregation done. Converting to DataFrame...
✅ Sociodemographic proportions saved at: /n/netscratch/cga/Lab/rpalbuquerque/Mobility/aggregated_aug2024/mobility_sociodemographics.csv.gz


In [ ]:
##⚙️ Bloco 6: Sociodemographic Aggregation by Sector